# L03: Perceptron animation

## Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/Courses/AI/Lecture01'

/content/drive/MyDrive/Courses/AI/Lecture01


In [ ]:
ls

perceptron-animation.ipynb  perceptron-numpy.ipynb  perceptron_toydata.txt


In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Preparing a toy dataset

In [4]:
##########################
### DATASET
##########################

data = np.genfromtxt('perceptron_toydata.txt', delimiter='\t')
X, y = data[:, :2], data[:, 2]
y = y.astype(int)
y[y==0] = -1

print('Class label counts:')
print(np.unique(y, return_counts=True))
print('X.shape:', X.shape)
print('y.shape:', y.shape)

# Shuffling & train/test split
shuffle_idx = np.arange(y.shape[0])
shuffle_rng = np.random.RandomState(123)
shuffle_rng.shuffle(shuffle_idx)
X, y = X[shuffle_idx], y[shuffle_idx]

X_train, X_test = X[shuffle_idx[:70]], X[shuffle_idx[70:]]
y_train, y_test = y[shuffle_idx[:70]], y[shuffle_idx[70:]]

# Normalize (mean zero, unit variance)
mu, sigma = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma



Class label counts:
(array([-1,  1]), array([50, 50]))
X.shape: (100, 2)
y.shape: (100,)


## Defining the Perceptron model

In [14]:
class Perceptron:
    def __init__(self, num_features, learning_rate=0.01):
        self.num_features = num_features
        self.weights = np.zeros((num_features, 1), dtype=float)
        self.bias = np.zeros(1, dtype=float)
        self.learning_rate = learning_rate

    def forward(self, x):
        linear = np.dot(x,self.weights) + self.bias  # Compute net input
        predictions = np.where(linear > 0., 1, -1)
        return predictions

    def backward(self, x, y):
        linear = np.dot(x,self.weights) + self.bias  # Compute net input
        cost = -y*linear
        if cost<0:
          errors = 0
        else:
          errors = 1
        grad_w = -1*errors*y*x
        grad_b = -1*errors*y
        return grad_w, grad_b

    def train(self, x, y, epochs):
        for e in range(epochs):
            for i in range(y.shape[0]):
                grad_w, grad_b = self.backward(x[i].reshape(1,self.num_features), y[i])
                self.weights -= grad_w.reshape(self.num_features, 1)
                self.bias -= grad_b

    def evaluate(self, x, y):
        predictions = self.forward(x).reshape(-1)
        accuracy = np.sum(predictions == y) / y.shape[0]
        return accuracy

## Training the Perceptron

In [15]:
all_weights = []
all_biases = []

ppn = Perceptron(num_features=2)

acc = 0

for epoch in range(10):

    for i in range(X.shape[0]):

        all_weights.append(ppn.weights.copy())
        all_biases.append(ppn.bias.copy())
        ppn.train(X[i].reshape(1, -1), y[i].reshape(-1), epochs=1)

        acc = ppn.evaluate(X, y)

        if acc == 1.0:
            break

    if acc == 1.0:
        all_weights.append(ppn.weights.copy())
        all_biases.append(ppn.bias.copy())
        break

## Plotting the boundary

In [16]:
##########################
### 2D Decision Boundary
##########################


import imageio


scatter_highlight_defaults = {'edgecolor': 'k',
                              'alpha': 1.0,
                              'linewidths': 2,
                              'marker': 'o',
                              's': 150}

def plot(i):

    fig, ax = plt.subplots()
    w, b = all_weights[i], all_biases[i]

    x_min = -20
    y_min = ( (-(w[0] * x_min) - b[0])
              / w[1] )

    x_max = 20
    y_max = ( (-(w[0] * x_max) - b[0])
              / w[1] )

    ax.set_xlim([-5., 5])
    ax.set_ylim([-5., 5])

    ax.set_xlabel('Iteration %d' % i)

    ax.plot([x_min, x_max], [y_min, y_max], color='k')

    ax.scatter(X[y==-1, 0], X[y==-1, 1], label='class -1', marker='o')
    ax.scatter(X[y==1, 0], X[y==1, 1], label='class 1', marker='s')

    ax.scatter(X[i][0], X[i][1], **scatter_highlight_defaults)

    fig.canvas.draw();
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    return image


kwargs_write = {'fps':1.0, 'quantizer':'nq'}
imageio.mimsave('training.gif', [plot(i) for i in range(len(all_weights))], fps=1);


Output hidden; open in https://colab.research.google.com to view.

In [17]:
pnn

NameError: name 'pnn' is not defined